# Pandas Merge

## Overview
My AT&T WiFi router has status information in HTML tables on two different pages. The common feature of the two tables is the IP address. The first table has the name associated with the device, while the second has the MAC address. I want to combine the information from both tables, and also get the manufacturer of the network adapter, if available. We can get that from the MAC address, using an online resource.

The steps involved are to
* Read data from the first table
 * Tidy up the data a bit
* Read data from the second table
* Merge the tables
* Add a column with the network adapter manufacturer
* Output to a CSV file (or any other desired format)

## Standard imports

In [1]:
# %load imports.py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline
import seaborn as sns
sns.set()



## Read first table

Read all the tables on the first router page, grabbing only tables whose text contain "Roku"

In [2]:
ROUTER_URL_1 = 'http://192.168.1.254/cgi-bin/home.ha'
ROUTER_MATCH_1 = "Roku"
DEVICE_LIST_1_DROP_LIST = ['Device IP Address / Name', ]  # columns we don't need

In [3]:
df_list = pd.read_html(
    ROUTER_URL_1, 
    match=ROUTER_MATCH_1,
    
)
device_list_1 = df_list.pop()

Split the "Device IP Address / Name" column into two new columns, and drop the original. 

In [4]:
device_list_1[['IP', 'name']] = device_list_1['Device IP Address / Name'].str.split(' / ', expand=True)
device_list_1.drop(DEVICE_LIST_1_DROP_LIST, axis=1, inplace=True)
device_list_1.columns  = 'status conn freq_type_name mesh IP name'.split()
device_list_1

,status,conn,freq_type_name,mesh,IP,name
0,on,Wi-Fi,"5 GHz, Home, CJMod",No,192.168.1.103,DEV6E5E42
1,on,Wi-Fi,"2.4 GHz, Home, CJMod",No,192.168.1.145,RokuPremiere
2,on,Ethernet,NaN,No,192.168.1.148,LivingRmAppleTV
3,off,Ethernet,NaN,No,192.168.1.157,LGwebOSTV
4,on,Wi-Fi,"2.4 GHz, Home, CJMod",No,192.168.1.162,KitchenTV
5,on,Wi-Fi,"2.4 GHz, Home, CJMod",No,192.168.1.165,BRW184F32B3125D
6,on,Wi-Fi,"5 GHz, Home, CJMod",No,192.168.1.216,ATT_4920_000E0C
7,on,Wi-Fi,"2.4 GHz, Home, CJMod",No,192.168.1.217,unknown18742e42e650
8,on,Wi-Fi,"2.4 GHz, Home, CJMod",Yes,192.168.1.218,BR
9,on,Wi-Fi,"5 GHz, Home, CJMod",No,192.168.1.222,V40-ThinQ


## Read second table

Read all the tables on the first router page, grapping only tables whose text contain "Authentication"

In [5]:
ROUTER_URL_2 = 'http://192.168.1.254/cgi-bin/lanstatistics.ha'
ROUTER_MATCH_2 = "Authentication"

In [6]:
df_list = pd.read_html(ROUTER_URL_2, match=ROUTER_MATCH_2)

device_list_2 = df_list.pop()
device_list_2.columns = 'mac auth IP access_point tx_pckts rx_pckts tx_bytes rx_bytes trans_err sig_str disassoc deauth'.split()
device_list_2

,mac,auth,IP,access_point,tx_pckts,rx_pckts,tx_bytes,rx_bytes,trans_err,sig_str,disassoc,deauth
0,d8:31:34:cf:2d:cc,Connected,192.168.1.145,2.4 GHz CJMod,64424.0,81487.0,1.313144e+09,19100577.0,1.0,-25 dBm,1.0,0.0
1,18:4f:32:b3:12:5d,Connected,192.168.1.165,2.4 GHz CJMod,8.0,620894.0,1.280524e+09,21243245.0,0.0,-55 dBm,4.0,0.0
2,c8:3a:6b:a5:7a:1a,Connected,192.168.1.162,2.4 GHz CJMod,1037609.0,126447.0,3.001580e+09,24489210.0,81.0,-54 dBm,5.0,0.0
3,00:d0:2d:bf:fe:2d,Connected,192.168.1.223,2.4 GHz CJMod,135568.0,170912.0,1.303504e+09,18064098.0,2117.0,-61 dBm,2.0,0.0
4,18:74:2e:42:e6:50,Connected,192.168.1.217,2.4 GHz CJMod,61685.0,3701724.0,1.300470e+09,143838234.0,58.0,-21 dBm,4.0,0.0
5,14:c1:4e:1b:01:ad,Connected,192.168.1.225,5 GHz CJMod,95500.0,49813.0,8.581194e+07,10949357.0,0.0,-42 dBm,7.0,7.0
6,f4:17:b8:00:0e:0c,Connected,192.168.1.216,5 GHz CJMod,NaN,NaN,NaN,NaN,NaN,-71 dBm,NaN,NaN
7,90:9c:4a:ce:e9:13,Connected,192.168.1.242,5 GHz CJMod,143619.0,86983.0,4.834786e+07,19877514.0,0.0,-55 dBm,0.0,0.0
8,ae:a5:79:5e:37:87,Connected,192.168.1.239,5 GHz CJMod,NaN,NaN,NaN,NaN,NaN,0 dBm,NaN,NaN
9,0c:84:dc:a4:36:47,Connected,192.168.1.103,5 GHz CJMod,32075.0,9528.0,1.003188e+07,1558119.0,0.0,-63 dBm,0.0,0.0


In [7]:
device_list_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   mac           14 non-null     object 
 1   auth          14 non-null     object 
 2   IP            14 non-null     object 
 3   access_point  14 non-null     object 
 4   tx_pckts      10 non-null     float64
 5   rx_pckts      10 non-null     float64
 6   tx_bytes      10 non-null     float64
 7   rx_bytes      10 non-null     float64
 8   trans_err     10 non-null     float64
 9   sig_str       14 non-null     object 
 10  disassoc      10 non-null     float64
 11  deauth        10 non-null     float64
dtypes: float64(7), object(5)
memory usage: 1.4+ KB


## Merge the tables
Now merge the tables, using the IP address as the common column on which to align rows. Since the columns have the same name in both tables, we can just use `on=COLUMN_NAME`. If the columns had different names, we could use `on_left=LEFT_COLUMN` and `on_right=RIGHT_COLUMN`.

We set `how` to `outer` so we get all rows from both dataframes. Otherwise, `how` defaults to `inner`, which only puts rows where the common column is in both dataframes in the result. 

In [8]:
merged_list = pd.merge(device_list_1, device_list_2, on="IP", how="outer")
merged_list

,status,conn,freq_type_name,mesh,IP,name,mac,auth,access_point,tx_pckts,rx_pckts,tx_bytes,rx_bytes,trans_err,sig_str,disassoc,deauth
0,on,Wi-Fi,"5 GHz, Home, CJMod",No,192.168.1.103,DEV6E5E42,0c:84:dc:a4:36:47,Connected,5 GHz CJMod,32075.0,9528.0,1.003188e+07,1558119.0,0.0,-63 dBm,0.0,0.0
1,on,Wi-Fi,"2.4 GHz, Home, CJMod",No,192.168.1.145,RokuPremiere,d8:31:34:cf:2d:cc,Connected,2.4 GHz CJMod,64424.0,81487.0,1.313144e+09,19100577.0,1.0,-25 dBm,1.0,0.0
2,on,Ethernet,NaN,No,192.168.1.148,LivingRmAppleTV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,off,Ethernet,NaN,No,192.168.1.157,LGwebOSTV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,on,Wi-Fi,"2.4 GHz, Home, CJMod",No,192.168.1.162,KitchenTV,c8:3a:6b:a5:7a:1a,Connected,2.4 GHz CJMod,1037609.0,126447.0,3.001580e+09,24489210.0,81.0,-54 dBm,5.0,0.0
5,on,Wi-Fi,"2.4 GHz, Home, CJMod",No,192.168.1.165,BRW184F32B3125D,18:4f:32:b3:12:5d,Connected,2.4 GHz CJMod,8.0,620894.0,1.280524e+09,21243245.0,0.0,-55 dBm,4.0,0.0
6,on,Wi-Fi,"5 GHz, Home, CJMod",No,192.168.1.216,ATT_4920_000E0C,f4:17:b8:00:0e:0c,Connected,5 GHz CJMod,NaN,NaN,NaN,NaN,NaN,-71 dBm,NaN,NaN
7,on,Wi-Fi,"2.4 GHz, Home, CJMod",No,192.168.1.217,unknown18742e42e650,18:74:2e:42:e6:50,Connected,2.4 GHz CJMod,61685.0,3701724.0,1.300470e+09,143838234.0,58.0,-21 dBm,4.0,0.0
8,on,Wi-Fi,"2.4 GHz, Home, CJMod",Yes,192.168.1.218,BR,8c:49:62:a9:00:a3,Connected,5 GHz CJMod,NaN,NaN,NaN,NaN,NaN,0 dBm,NaN,NaN
9,on,Wi-Fi,"5 GHz, Home, CJMod",No,192.168.1.222,V40-ThinQ,64:c2:de:79:f3:b0,Connected,5 GHz CJMod,91725.0,56433.0,6.668618e+07,16614144.0,0.0,-51 dBm,5.0,5.0


## Add column with network adapter manufacturer
We will use the MAC address column to fetch the manufacturer, use the **requests** library. The `apply()` method can be used to apply a user-defined function to a series or dataframe. 

In [11]:
MAC_VENDOR_URL = "http://api.macvendors.com/"
import requests
import time

def get_mfr_by_mac(row):
    mac = row['mac']
    if not isinstance(mac, str):
        return "N/A"
    url = MAC_VENDOR_URL + mac
    response = requests.get(url)
    if response.status_code == requests.codes.OK:
        mfr = response.text
    else:
        mfr = 'UNKNOWN'
    time.sleep(1.1)  # don't over-request from API
    return mfr

In [12]:
merged_list['mfr'] = merged_list.apply(get_mfr_by_mac, axis=1)
merged_list


,status,conn,freq_type_name,mesh,IP,name,mac,auth,access_point,tx_pckts,rx_pckts,tx_bytes,rx_bytes,trans_err,sig_str,disassoc,deauth,mfr
0,on,Wi-Fi,"5 GHz, Home, CJMod",No,192.168.1.103,DEV6E5E42,0c:84:dc:a4:36:47,Connected,5 GHz CJMod,32075.0,9528.0,1.003188e+07,1558119.0,0.0,-63 dBm,0.0,0.0,"Hon Hai Precision Ind. Co.,Ltd."
1,on,Wi-Fi,"2.4 GHz, Home, CJMod",No,192.168.1.145,RokuPremiere,d8:31:34:cf:2d:cc,Connected,2.4 GHz CJMod,64424.0,81487.0,1.313144e+09,19100577.0,1.0,-25 dBm,1.0,0.0,"Roku, Inc"
2,on,Ethernet,NaN,No,192.168.1.148,LivingRmAppleTV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N/A
3,off,Ethernet,NaN,No,192.168.1.157,LGwebOSTV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N/A
4,on,Wi-Fi,"2.4 GHz, Home, CJMod",No,192.168.1.162,KitchenTV,c8:3a:6b:a5:7a:1a,Connected,2.4 GHz CJMod,1037609.0,126447.0,3.001580e+09,24489210.0,81.0,-54 dBm,5.0,0.0,"Roku, Inc"
5,on,Wi-Fi,"2.4 GHz, Home, CJMod",No,192.168.1.165,BRW184F32B3125D,18:4f:32:b3:12:5d,Connected,2.4 GHz CJMod,8.0,620894.0,1.280524e+09,21243245.0,0.0,-55 dBm,4.0,0.0,"Hon Hai Precision Ind. Co.,Ltd."
6,on,Wi-Fi,"5 GHz, Home, CJMod",No,192.168.1.216,ATT_4920_000E0C,f4:17:b8:00:0e:0c,Connected,5 GHz CJMod,NaN,NaN,NaN,NaN,NaN,-71 dBm,NaN,NaN,AirTies Wireless Networks
7,on,Wi-Fi,"2.4 GHz, Home, CJMod",No,192.168.1.217,unknown18742e42e650,18:74:2e:42:e6:50,Connected,2.4 GHz CJMod,61685.0,3701724.0,1.300470e+09,143838234.0,58.0,-21 dBm,4.0,0.0,Amazon Technologies Inc.
8,on,Wi-Fi,"2.4 GHz, Home, CJMod",Yes,192.168.1.218,BR,8c:49:62:a9:00:a3,Connected,5 GHz CJMod,NaN,NaN,NaN,NaN,NaN,0 dBm,NaN,NaN,"Roku, Inc"
9,on,Wi-Fi,"5 GHz, Home, CJMod",No,192.168.1.222,V40-ThinQ,64:c2:de:79:f3:b0,Connected,5 GHz CJMod,91725.0,56433.0,6.668618e+07,16614144.0,0.0,-51 dBm,5.0,5.0,LG Electronics (Mobile Communications)


## Export the data

In [15]:
merged_list.to_csv("network_info.csv")
merged_list.to_excel('network_info.xlsx')
merged_list.to_json('network_info.json')
merged_list.to_html('network_info.html')
merged_list.to_xml('network_info.xml')